### Things to do
##### Differernt Models
#### 1. Gradient Boosting Classifier
#### 2. Isolation classifier
#### 3. Extra Tree classifier
#### 4. Voting classifier
##### Feature Selection/Engineering
##### ANN

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train['Hillshade_mean'] = train['Hillshade_9am']+train['Hillshade_Noon']+train['Hillshade_3pm']
train['slope_hyd'] = (train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)**0.5
train.slope_hyd=train.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any
train['log_elevation']= np.log(train.Elevation)
train['Hillshade_9am_sq'] = train['Hillshade_9am']**2
train['Hillshade_noon_sq'] = train['Hillshade_Noon']**2
train['Hillshade_3pm_sq'] = train['Hillshade_3pm']**2

train['interaction_9amnoon'] = train['Hillshade_9am']*train['Hillshade_Noon']
train['interaction_noon3pm'] = train['Hillshade_3pm']*train['Hillshade_Noon']
train['interaction_9am3pm'] = train['Hillshade_3pm']*train['Hillshade_9am']

In [5]:
test['Hillshade_mean'] = test['Hillshade_9am']+test['Hillshade_Noon']+test['Hillshade_3pm']
test['slope_hyd'] = (test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)**0.5
test.slope_hyd=test.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any
test['log_elevation']= np.log(test.Elevation)
test['Hillshade_9am_sq'] = test['Hillshade_9am']**2
test['Hillshade_noon_sq'] = test['Hillshade_Noon']**2
test['Hillshade_3pm_sq'] = test['Hillshade_3pm']**2

test['interaction_9amnoon'] = test['Hillshade_9am']*test['Hillshade_Noon']
test['interaction_noon3pm'] = test['Hillshade_3pm']*test['Hillshade_Noon']
test['interaction_9am3pm'] = test['Hillshade_3pm']*test['Hillshade_9am']

In [6]:
X_train = train.drop(['Id','Cover_Type'],1)
Y_train = train['Cover_Type']
X_test = test.drop(['Id'],1)

In [7]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [8]:
X_train_tf.head(2)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type40,Hillshade_mean,slope_hyd,log_elevation,Hillshade_9am_sq,Hillshade_noon_sq,Hillshade_3pm_sq,interaction_9amnoon,interaction_noon3pm,interaction_9am3pm
0,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805,...,-0.176939,0.666456,0.102333,-0.297134,0.221026,0.562730,0.128799,0.590554,0.353160,0.690441
1,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710,...,-0.176939,0.763782,-0.110745,-0.312188,0.184436,0.709867,0.203436,0.644256,0.451832,0.760204


## Light GBM

In [9]:
print(np.unique(Y_train))
Y_train_temp=Y_train-1
print(np.unique(Y_train_temp))

[1 2 3 4 5 6 7]
[0 1 2 3 4 5 6]


In [23]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train_tf,label=Y_train_temp)
params = {}
params['learning_rate'] = 0.003
params['device'] = 'cpu'
params['boosting_type'] = 'dart'
#params['boosting_type'] = 'rf'
#params['boosting_type'] = 'gbdt'
params['application'] = 'multiclass'
params['num_class']= 7
params['metric'] = 'multi_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 30
params['min_data'] = 100
params['max_depth'] = 15
clf = lgb.train(params, d_train, 500)

In [24]:
Y_pred=clf.predict(X_train_tf)
Y_pred

array([[ 0.11935021,  0.11117444,  0.1482719 , ...,  0.13366854,
         0.15907605,  0.10805809],
       [ 0.12138327,  0.11330125,  0.13065885, ...,  0.13622568,
         0.13431376,  0.11012528],
       [ 0.11609037,  0.12553625,  0.1465681 , ...,  0.12840116,
         0.15739813,  0.1074599 ],
       ..., 
       [ 0.11072121,  0.11016758,  0.16280062, ...,  0.11770719,
         0.17154843,  0.10894136],
       [ 0.11412078,  0.1135917 ,  0.13893737, ...,  0.12136564,
         0.15753584,  0.11232736],
       [ 0.11105628,  0.10974667,  0.16163791, ...,  0.11709511,
         0.1761041 ,  0.10894941]])

In [25]:
Y_pred_max=np.argmax(Y_pred,axis=1)
np.unique(Y_pred_max)

array([3])

In [16]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(Y_pred_max,Y_train_temp)
accuracy

0.79576719576719579

## Randon Forrest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [20]:
results_rf = pd.DataFrame(data=None,columns=['n_estimator','criterion','max_depth',
                                            'accuracy_min','accuracy_max','accuracy_mean'])

In [21]:
N_estimators = [20,40,60,80]
Criterion = ['gini','entropy']
Max_depth = [20,30,40,50,60,70]
i=0
for n_estimator in N_estimators:
    for criterion in Criterion:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'criterion'] = criterion
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion=criterion,
                                                      max_depth=max_depth,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1

In [22]:
results_rf

,n_estimator,criterion,max_depth,accuracy_min,accuracy_max,accuracy_mean
0,20,gini,20,0.712302,0.814153,0.752513
1,20,gini,30,0.713294,0.823413,0.752712
2,20,gini,40,0.70668,0.820437,0.75086
3,20,gini,50,0.70668,0.820437,0.75086
4,20,gini,60,0.70668,0.820437,0.75086
5,20,gini,70,0.70668,0.820437,0.75086
6,20,entropy,20,0.718254,0.812831,0.754431
7,20,entropy,30,0.711971,0.818452,0.75086
8,20,entropy,40,0.71131,0.822421,0.750066
9,20,entropy,50,0.71131,0.822421,0.750066
